In [94]:
import pandas as pd
import numpy as np

In [95]:
df_social_networks = pd.read_csv('task_3_data.csv')
df_social_networks

,date,media_souce,cost,installs,activations,expected_revenue
0,2023-11-01,Meta,3189.69,959,175,3401.7000
1,2023-11-02,Meta,3062.85,1071,161,4303.6100
2,2023-11-03,Meta,3116.61,994,189,3210.6500
3,2023-11-04,Meta,3111.15,1022,175,3583.8500
4,2023-11-05,Meta,3297.28,1157,161,3346.8800
5,2023-11-06,Meta,3365.81,1064,182,3443.4100
6,2023-11-07,Meta,3085.32,1024,189,3701.4800
7,2023-11-08,Meta,3220.14,1084,182,4102.3200
8,2023-11-09,Meta,3174.43,1057,182,4063.8400
9,2023-11-10,Meta,3133.27,1011,189,3536.2200


Checking if any 0 values in the dataframe

In [96]:
df_social_networks.isin([0, "0"]).values.any()

np.False_

Checking for datatypes, and NaN values. 
P.S. since dataframe is small, I won't change data types to 32-16 bit

In [97]:
df_social_networks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              60 non-null     object 
 1   media_souce       60 non-null     object 
 2   cost              60 non-null     float64
 3   installs          60 non-null     int64  
 4   activations       60 non-null     int64  
 5   expected_revenue  60 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.9+ KB


In [98]:
df_social_networks

,date,media_souce,cost,installs,activations,expected_revenue
0,2023-11-01,Meta,3189.69,959,175,3401.7000
1,2023-11-02,Meta,3062.85,1071,161,4303.6100
2,2023-11-03,Meta,3116.61,994,189,3210.6500
3,2023-11-04,Meta,3111.15,1022,175,3583.8500
4,2023-11-05,Meta,3297.28,1157,161,3346.8800
5,2023-11-06,Meta,3365.81,1064,182,3443.4100
6,2023-11-07,Meta,3085.32,1024,189,3701.4800
7,2023-11-08,Meta,3220.14,1084,182,4102.3200
8,2023-11-09,Meta,3174.43,1057,182,4063.8400
9,2023-11-10,Meta,3133.27,1011,189,3536.2200


Cutting objects (data/string) off and checking numbers correlations

In [99]:
df_social_networks.iloc[:, 2:].corr()

,cost,installs,activations,expected_revenue
cost,1.000000,0.932881,0.963749,0.947808
installs,0.932881,1.000000,0.898338,0.868565
activations,0.963749,0.898338,1.000000,0.945999
expected_revenue,0.947808,0.868565,0.945999,1.000000


Calculating CPI (cost per install $), activation rate(%) and potential ROI (return on investment %)

In [104]:
df_social_networks["CPI"] = df_social_networks["cost"] / df_social_networks["installs"]
df_social_networks["activation_rate"] = (
    df_social_networks["activations"] / df_social_networks["installs"]
) * 100
df_social_networks["ROI"] = (
    (
        df_social_networks["expected_revenue"] - df_social_networks["cost"]
    ) / df_social_networks["cost"]
) * 100

Grouping data by media source

In [105]:
grouped_metrics = df_social_networks.groupby("media_souce").agg({
    "CPI": "mean",
    "activation_rate": "mean",
    "ROI": "mean"
}).round(4)
grouped_metrics

,CPI,activation_rate,ROI
media_souce,,,
Meta,3.4928,20.8274,23.4318
Tiktok,3.4138,16.0257,16.0845


Unload data to CSV

In [106]:
grouped_metrics.to_csv('grouped_metrics.csv')